In [24]:
import pandas as pd
import numpy as np
import geopandas as gpd
import leafmap as leafmap
import dash_leaflet as dl #from tutorial, replace with leafmap later
#dcc = dash_core_components
#hmtl = dash_html_components
from dash import Dash, html, dcc, callback, Output, Input, dash_table
import plotly.express as px
import json


In [25]:

geom_gdf = gpd.read_file('data/terrawise/Stratification/1/cea part/cea_1.shp')
geom_unify = geom_gdf.dissolve()
geom_unify['label']=1
centroid_geom1 = geom_gdf.centroid



In [26]:
practice_csv = pd.read_csv('data/terrawise/terrawise-practice-info.csv', usecols=['label','Year', 'Land use', 'Urea', 'Lime','Synthetic_fertiliser','Biochar'])
geom_practice_data = practice_csv[practice_csv['label']==1]

In [27]:
#attribute join between CEA's and practice data assuming 'label' corresponds to CEA
geom_combined = geom_unify.merge(geom_practice_data, on='label')
geom_geojson=json.loads(geom_combined.to_json(to_wgs84=True))


In [28]:

app = Dash(__name__)

app.layout = html.Div([
    html.Div(children='Practice management data'),
    html.Hr(),
    dash_table.DataTable(data=practice_csv.to_dict('records'), page_size=3),
    dcc.Dropdown(practice_csv.columns, id='controls-practice-dropdown'),
    dcc.Graph(figure={}, id='graph-by-practice'),
    dcc.Dropdown(practice_csv['Year'], id='controls-year-dropdown'),
    dcc.Graph(figure={}, id='graph-by-year'),
    html.Div([
        html.Div(children='Mapped data (not currently linked)'),
        html.Hr(),
        dl.Map(children=[
                dl.TileLayer(),
                dl.GeoJSON(data=geom_geojson,
                    zoomToBounds=True,
                    id='shapes')],
                id='map',
                style={'height': '40vh'},
                center=[-31, 120], #replace with centroid and extent of geometry data
                zoom=4,
                zoomControl=True,
                doubleClickZoom=False,
                dragging=False,
                scrollWheelZoom=True,
                preferCanvas=True)],
            id='map_div', className='map_div')])

@callback(
    Output(component_id='graph-by-practice', component_property='figure'),
    Input(component_id='controls-practice-dropdown', component_property='value')
)
@callback(
    Output(component_id='graph-by-year', component_property='figure'),
    Input(component_id='controls-year-dropdown', component_property='value')
)


def update_graph(col_chosen):
    fig = px.histogram(practice_csv, x=col_chosen, y='label', histfunc='count')
    return fig



if __name__ == '__main__':
    app.run(debug=True, port=8051)

---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
File ~/miniconda3/envs/geoprocessing-gdal-cogs/lib/python3.11/site-packages/plotly/express/_chart_types.py:480, in histogram(
    data_frame=    label  Year        Land use Lime Urea Biocha...      Crop    N    Y       N                    Y,
    x=2022,
    y='label',
    color=None,
    pattern_shape=None,
    facet_row=None,
    facet_col=None,
    facet_col_wrap=0,
    facet_row_spacing=None,
    facet_col_spacing=None,
    hover_name=None,
    hover_data=None,
    animation_frame=None,
    animation_group=None,
    category_orders=None,
    labels=None,
    color_discrete_sequence=None,
    color_discrete_map=None,
    pattern_shape_sequence=None,
    pattern_shape_map=None,
    marginal=None,
    opacity=None,
    orientation=None,
    barmode='relative',
    barnorm=None,
    histnorm=None,
    log_x=False,
    log_y=False,
    r